## Precarga de librerias y funciones

In [24]:
%%capture
%run "../../recurrentes.ipynb"

In [25]:
%%capture
%run "../../funciones.ipynb"

#### Excel

In [26]:
# Activar Excel de seguimiento de medidas? 

excel = True
if excel == True:
    # Ingresar path excel para anotar resultados
    path_excel = base_path_l + r'\resultados_backtesting.xlsx'
    date = datetime.now().strftime("%d-%m-%Y %H-%M-%S")
    

In [27]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) competencia_03_ct_sin_psi.parquet

dataset = dataset_competencia_03_lags_deltas_y_clase_ternaria
dataset_name = os.path.basename(dataset)
df_train = pd.read_parquet(dataset)


In [28]:
columnas = df_train.columns
columnas_lags = [col for col in columnas if 'lag' in col]

In [29]:
df_train = df_train.drop(columns=columnas_lags)

In [30]:
df_train.shape

(4901237, 320)

In [ ]:
columnas_num = df_train.select_dtypes(exclude=['bool','object'])
lista_columnas_num = columnas_num.columns

In [ ]:
df_train['max'] = df_train[lista_columnas_num].max(axis=1)

In [ ]:
# Lista de variables a verificar
variables = [
    'cmobile_app_trx', 'Master_cconsumos', 'ctarjeta_debito_transacciones', 
    'ctarjeta_visa_transacciones', 'ctarjeta_master_transacciones', 'cpayroll_trx', 
    'cpayroll2_trx', 'ccuenta_debitos_automaticos', 'cpagodeservicios', 'cforex', 
    'cforex_buy', 'mforex_sell', 'cextraccion_autoservicio', 'ccallcenter_transacciones', 
    'chomebanking_transacciones'
]

# Verificar si 'ctrx_quarter' existe en el DataFrame
if 'ctrx_quarter' not in df_train.columns:
    raise KeyError("La variable 'ctrx_quarter' no existe en el dataset.")

# Lista de las variables que sí están presentes en el DataFrame
variables_presentes = [var for var in variables if var in df_train.columns]

# Si hay variables presentes en el DataFrame, calcular la Power Ratio
if variables_presentes:
    for var in variables_presentes:
        # Evitar división por cero reemplazando ceros en el denominador por un valor mínimo (1e-6)
        denominator = df_train[var].replace(0, 1e-6) ** 2  # Elevar al cuadrado el denominador
        # Crear una nueva columna con el prefijo 'power_ratio_'
        df_train['power_ratio_' + var] = df_train['ctrx_quarter'] / denominator
else:
    print("No se encontraron variables para calcular la Power Ratio.")

# Informar sobre las variables que faltan en el DataFrame
variables_faltantes = [var for var in variables if var not in df_train.columns]
if variables_faltantes:
    print(f"Las siguientes variables no existen en el dataset y no se calcularon: {variables_faltantes}")
else:
    print("Todas las variables fueron verificadas correctamente.")


In [ ]:
columnas_tarjeta = df_train.columns
columnas_tarjeta= [col for col in columnas_tarjeta if 'conjunto' in col]
columnas_tarjeta


In [ ]:
df_train.shape

In [ ]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

# Lista de columnas a las que se aplicará PolynomialFeatures
columnas_polynomial = ['ctrx_quarter', 'conjunto_Visa_msaldototal', 'cpayroll_trx']

# Verificar que las columnas existen en df_train
for col in columnas_polynomial:
    if col not in df_train.columns:
        raise KeyError(f"La columna '{col}' no existe en df_train.")

# Extraer las columnas seleccionadas
df_poly = df_train[columnas_polynomial].copy()

# Configurar PolynomialFeatures
grado_polinomio = 3  # Cambia este valor según tus necesidades
poly = PolynomialFeatures(degree=grado_polinomio, include_bias=False, interaction_only=True)

# Ajustar y transformar las características para generar las nuevas características polinomiales
poly_features = poly.fit_transform(df_poly)

# Obtener los nombres de las nuevas características generadas
nombres_columnas = poly.get_feature_names_out(columnas_polynomial)

# Crear un DataFrame con las nuevas características polinomiales
df_poly_features = pd.DataFrame(poly_features, columns=nombres_columnas, index=df_poly.index)

# Concatenar las nuevas características al DataFrame original
df_train_expanded = pd.concat([df_train, df_poly_features], axis=1)

# Opcional: Si deseas eliminar las columnas originales después de la expansión
# df_train_expanded = df_train_expanded.drop(columns=columnas_polynomial)

# Resultado final en df_train_expanded con las nuevas características


In [ ]:
df_train_expanded = df_train_expanded.drop(columns=columnas_polynomial)

df_train_expanded.shape

In [ ]:
df_train = df_train_expanded

## Formateo pre modelo

In [ ]:

# [202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106] modelo 10 meses

ganancia_acierto = 273000
costo_estimulo = 7000

#Defenir mes de train y test

mes_train = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202010, 202011, 202012,
       202101, 202102, 202103, 202104, 202105]
mes_test = 202107


# Defenir mejores parámetros para el modelo

best_params = {'num_leaves': 94,
 'learning_rate': 0.050129899443463674,
 'min_data_in_leaf': 1004,
 'feature_fraction': 0.2530957925821187,
 'bagging_fraction': 0.28656832468240556}
best_iter = 300

In [ ]:
# predecir_punto_de_corte(  
#     dataset,  
#     mes_de_entrenamiento,  
#     mes_de_prueba,  
#     best_params,  
#     best_iteration,  
#     rango_cortes,  
#     semillas,  


In [ ]:

data = df_train
data['tmobile_app'] = pd.to_numeric(data['tmobile_app'], errors='coerce')  # Convierte no numéricos a NaN
data['tmobile_app'] = data['tmobile_app'].fillna(0).astype('bool')  # Llena los NaN y convierte a booleano


data['cmobile_app_trx'] = pd.to_numeric(data['cmobile_app_trx'], errors='coerce')  # Convierte no numéricos a NaN
data['cmobile_app_trx'] = data['cmobile_app_trx'].fillna(0).astype('bool')  # Llena los NaN y convierte a booleano

In [ ]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)
    
# Binarización de la variable target

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='BAJA+2', 1, 0)

# Train-Test Split

if isinstance(mes_train, list):
    df_train = data[data['foto_mes'].isin(mes_train)]
    
else: df_train = data[data['foto_mes']<=mes_train]

df_test = data[data['foto_mes']==mes_test]

# definiendo X e Y

clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train['clase_peso']
w_test = df_test['clase_peso']

In [ ]:
df_train['foto_mes'].unique(), df_test['foto_mes'].unique()

## Back-testing

In [ ]:
respuesta = 'personalizado'
if respuesta == '':
   input (" 'personalizado' o 'default': ")
   if respuesta == 'personalizado':
      
      best_params = {'num_leaves': '',
      'learning_rate': '',
      'min_data_in_leaf': '',
      'feature_fraction': '',
      'bagging_fraction': ''}
   elif respuesta == 'default':
      params = {}


In [ ]:
df_voting = pd.DataFrame()


train_data = lgb.Dataset(X_train, label=Y_train, weight=w_train)

for x in semillas:

    if respuesta == 'personalizado':
        params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'first_metric_only': True,
            'boost_from_average': True,
            'feature_pre_filter': False,
            'max_bin': 31,
            'num_leaves': best_params['num_leaves'],
            'learning_rate': best_params['learning_rate'],
            'min_data_in_leaf': best_params['min_data_in_leaf'],
            'feature_fraction': best_params['feature_fraction'],
            'bagging_fraction': best_params['bagging_fraction'],
            'seed': x,  # Usamos el x actual del bucle
            'verbose': 0
        }

    elif respuesta == 'default':
        params = {
            'seed': x  # Si deseas usar x también en el caso 'default'
        }

    # Entrenar el modelo con los parámetros actualizados
    model = lgb.train(params,
                      train_data,
                      num_boost_round=best_iter)

    y_pred = model.predict(X_test)
    df_voting[f'prediccion_seed_{x}'] = y_pred

df_voting['prediccion'] = df_voting.mean(axis=1)
df_voting.index = X_test.index


In [ ]:
prediccion = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'probabilidad': df_voting ['prediccion']}, index=X_test.index) 
prediccion.shape 

In [ ]:
prediccion.isna().sum()

In [ ]:
df_train['foto_mes'].unique()

In [ ]:
df_test = df_test.reindex(X_test.index)


In [ ]:
bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']}, index= X_test.index)
bajas.shape

In [ ]:
X_test.shape

In [ ]:
print(df_test['clase_ternaria'].isna().sum())


In [ ]:
df_test.shape

In [ ]:
bajas.isna().sum()

In [ ]:
prediccion.shape
merged_data = prediccion.merge(bajas, on='numero_de_cliente', how='inner')

merged_data.columns
merged_data_sorted = merged_data.sort_values('probabilidad', ascending=False)


# Ordenar por probabilidad de mayor a menor

# Seleccionar top clientes
top_clients = merged_data_sorted.iloc[:12000].copy()  # Trabajar con una copia

# Convertir 'clase_ternaria' a variable binaria para los top clientes
top_clients.loc[:, 'bajas_reales'] = (top_clients['clase_ternaria'] == 'BAJA+2').astype(int)

top_clients.value_counts('clase_ternaria')


In [ ]:
top_clients.value_counts('bajas_reales')